In [2]:
import pandas as pd
import numpy as ny
import sklearn as sk
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
from tools.data_tools import get_files, read_file

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.cross_validation import KFold


/home/lijing/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/lijing/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
g = get_files()

In [5]:
g

{'BalanceSheet-Bank.csv': '/home/lijing/.keras/fddc2018-01/BalanceSheet-Bank.csv',
 'BalanceSheet-GeneralBusiness.csv': '/home/lijing/.keras/fddc2018-01/BalanceSheet-GeneralBusiness.csv',
 'BalanceSheet-Insurance.csv': '/home/lijing/.keras/fddc2018-01/BalanceSheet-Insurance.csv',
 'BalanceSheet-Securities.csv': '/home/lijing/.keras/fddc2018-01/BalanceSheet-Securities.csv',
 'CashFlowStatement-Bank.csv': '/home/lijing/.keras/fddc2018-01/CashFlowStatement-Bank.csv',
 'CashFlowStatement-GeneralBusiness.csv': '/home/lijing/.keras/fddc2018-01/CashFlowStatement-GeneralBusiness.csv',
 'CashFlowStatement-Insurance.csv': '/home/lijing/.keras/fddc2018-01/CashFlowStatement-Insurance.csv',
 'CashFlowStatement-Securities.csv': '/home/lijing/.keras/fddc2018-01/CashFlowStatement-Securities.csv',
 'CompanyOperation.csv': '/home/lijing/.keras/fddc2018-01/CompanyOperation.csv',
 'IncomeStatement-Bank.csv': '/home/lijing/.keras/fddc2018-01/IncomeStatement-Bank.csv',
 'IncomeStatement-GeneralBusiness.csv'

In [6]:
def get_data(filename, flist, ticker_symbol, report_type):
    dataframe = read_file(filename)
    dataframe = dataframe[dataframe['TICKER_SYMBOL'] == ticker_symbol][flist].sort_values(by='END_DATE')
    dataframe = dataframe.drop_duplicates('END_DATE', keep='last').reset_index(drop=True)
    dataframe = dataframe[dataframe['REPORT_TYPE'] == report_type]
    
    return dataframe

In [37]:
cfs_data = get_data('CashFlowStatement-GeneralBusiness.csv',
                   ['TICKER_SYMBOL', 'REPORT_TYPE',
                    'END_DATE', 'C_FR_SALE_G_S',
                    'N_CHANGE_IN_CASH', 'N_CE_BEG_BAL', 'N_CE_END_BAL'],
                    '000002', 'S1').reset_index(drop=True)
bs_data = get_data('BalanceSheet-GeneralBusiness.csv',
                  ['TICKER_SYMBOL', 'REPORT_TYPE',
                   'END_DATE', 'CASH_C_EQUIV',
                   'T_EQUITY_ATTR_P', 'MINORITY_INT', 'T_SH_EQUITY',
                   'T_LIAB_EQUITY'],
                   '000002', 'S1').reset_index(drop=True)
is_data = get_data('IncomeStatement-GeneralBusiness.csv',
                  ['TICKER_SYMBOL', 'END_DATE', 'REPORT_TYPE', 'REVENUE'],
                  '000002', 'S1').reset_index(drop=True)

In [66]:

test_data = cfs_data[9:].drop(['TICKER_SYMBOL', 'REPORT_TYPE', 'END_DATE'], axis=1).reset_index(drop=True)
test_data = test_data.join(bs_data[8:].drop(['TICKER_SYMBOL', 'REPORT_TYPE', 'END_DATE'], axis=1).reset_index(drop=True))

test_data

,C_FR_SALE_G_S,N_CHANGE_IN_CASH,N_CE_BEG_BAL,N_CE_END_BAL,CASH_C_EQUIV,T_EQUITY_ATTR_P,MINORITY_INT,T_SH_EQUITY,T_LIAB_EQUITY
0,1.513991e+11,2.019314e+10,7.949001e+10,9.968316e+10,1.075632e+11,1.126407e+11,4.851710e+10,1.611578e+11,9.292659e+11


In [8]:
cfs_data = cfs_data.drop([0, 9]).reset_index(drop=True)
is_data = is_data.drop([0, 9]).reset_index(drop=True)
bs_data = bs_data.drop(8).reset_index(drop=True)

In [9]:
# from 2008-06-30 ~ 2016-06-30
y = is_data.drop(['TICKER_SYMBOL', 'END_DATE', 'REPORT_TYPE'], axis=1)

In [10]:
# create train data

train_data = cfs_data.drop(['TICKER_SYMBOL', 'REPORT_TYPE', 'END_DATE'], axis=1)
# train_data = train_data.append(bs_data.drop(['TICKER_SYMBOL', 'REPORT_TYPE', 'END_DATE'], axis=1))
train_data = train_data.join(bs_data.drop(['TICKER_SYMBOL', 'REPORT_TYPE', 'END_DATE'], axis=1))
train_data = train_data.join(y)

train_data

,C_FR_SALE_G_S,N_CHANGE_IN_CASH,N_CE_BEG_BAL,N_CE_END_BAL,CASH_C_EQUIV,T_EQUITY_ATTR_P,MINORITY_INT,T_SH_EQUITY,T_LIAB_EQUITY,REVENUE
0,2.732685e+10,6.902138e+09,1.997829e+10,2.688042e+10,2.688042e+10,3.452127e+10,7.283670e+09,4.180494e+10,1.245199e+11,2.180865e+10
1,2.887052e+10,-4.003392e+09,2.200277e+10,1.799938e+10,1.911131e+10,3.977295e+10,8.646855e+09,4.841981e+10,1.605127e+11,1.676633e+10
2,5.040683e+10,4.949835e+09,3.509694e+10,4.004677e+10,4.077947e+10,4.620453e+10,1.160295e+10,5.780748e+10,2.609603e+11,1.998884e+10
3,4.816428e+10,1.210562e+10,3.361411e+10,4.571973e+10,4.701219e+10,5.509982e+10,1.473416e+10,6.983397e+10,3.304012e+11,3.072299e+10
4,6.722044e+10,-1.487271e+10,5.112022e+10,3.624751e+10,3.760363e+10,6.664463e+10,2.142085e+10,8.806548e+10,4.322420e+11,4.139035e+10
5,6.522281e+10,-1.291825e+09,4.300415e+10,4.171232e+10,4.286172e+10,7.688847e+10,2.883141e+10,1.057199e+11,5.017743e+11,4.096190e+10
6,7.571239e+10,-1.788279e+10,6.165332e+10,4.377053e+10,4.461256e+10,8.727269e+10,3.052716e+10,1.177999e+11,5.369376e+11,5.026680e+10
7,1.273494e+11,1.915959e+10,5.174762e+10,7.090721e+10,7.186799e+10,9.746073e+10,4.071332e+10,1.381740e+11,7.123067e+11,7.479529e+10


In [ ]:
# colormap = plt.cm.RdBu
# plt.figure(figsize=(14,12))
# plt.title('Pearson Correlation of Features', y=1.05, size=15)
# sns.heatmap(train_data.astype(float).corr(),linewidths=0.1,vmax=1.0, 
#             square=True, cmap=colormap, linecolor='white', annot=True)

In [ ]:
y_train = train_data['REVENUE'].ravel()
train = train_data.drop(['REVENUE'], axis=1)
x_train = train.values # Creates an array of the train data
x_test = test.values # Creats an array of the test data